In [1]:
import json

In [2]:
with open('./documents.json', 'rt') as f_in:
    documents_file = json.load(f_in)
    print(f"Number of courses - {len(documents_file)}")

Number of courses - 3


In [3]:
documents_file[0]['documents'][0]

{'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.",
 'section': 'General course-related questions',
 'question': 'Course - When will the course start?'}

In [4]:
documents = []

for course in documents_file:
    course_name = course['course']

    for doc in course['documents']:
        doc['course'] = course_name
        documents.append(doc)

In [5]:
documents[0]
print(len(documents))

948


In [6]:
from elasticsearch import Elasticsearch

es = Elasticsearch("http://localhost:9200")
es.info()

ObjectApiResponse({'name': 'dce6cf3cc571', 'cluster_name': 'docker-cluster', 'cluster_uuid': 'BdoFPARTQS6dXg4li9Zs-g', 'version': {'number': '8.4.3', 'build_flavor': 'default', 'build_type': 'docker', 'build_hash': '42f05b9372a9a4a470db3b52817899b99a76ee73', 'build_date': '2022-10-04T07:17:24.662462378Z', 'build_snapshot': False, 'lucene_version': '9.3.0', 'minimum_wire_compatibility_version': '7.17.0', 'minimum_index_compatibility_version': '7.0.0'}, 'tagline': 'You Know, for Search'})

In [7]:
index_settings = {
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0
    },
    "mappings": {
        "properties": {
            "text": {"type": "text"},
            "section": {"type": "text"},
            "question": {"type": "text"},
            "course": {"type": "keyword"} 
        }
    }
}

index_name = "course-questions"
response = es.indices.create(index=index_name, body=index_settings)

response

BadRequestError: BadRequestError(400, 'resource_already_exists_exception', 'index [course-questions/a4_vWyZ7Q_2zMeLmC-S47w] already exists')

In [8]:
index_name = "course-questions"

In [9]:
from tqdm.auto import tqdm

for doc in tqdm(documents):
    es.index(index=index_name, document=doc)

/home/codespace/.local/share/virtualenvs/llm-rag-workshop-XedGi05j/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 948/948 [00:29<00:00, 31.66it/s]


In [11]:
user_question = "How do I join the course after it has started?"

search_query = {
    "size": 5,
    "query": {
        "bool": {
            "must": {
                "multi_match": {
                    "query": user_question,
                    "fields": ["question^3", "text", "section"],
                    "type": "best_fields"
                }
            },
            "filter": {
                "term": {
                    "course": "data-engineering-zoomcamp"
                }
            }
        }
    }
}

In [12]:
response = es.search(index=index_name, body=search_query)

for hit in response['hits']['hits']:
    doc = hit['_source']
    print(f"Section: {doc['section']}\nQuestion: {doc['question']}\nAnswer: {doc['text']}\n\n")

Section: General course-related questions
Question: Course - Can I still join the course after the start date?
Answer: Yes, even if you don't register, you're still eligible to submit the homeworks.
Be aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute.


Section: General course-related questions
Question: Course - Can I still join the course after the start date?
Answer: Yes, even if you don't register, you're still eligible to submit the homeworks.
Be aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute.


Section: General course-related questions
Question: Course - Can I follow the course after it finishes?
Answer: Yes, we will keep all the materials after the course finishes, so you can follow the course at your own pace after it finishes.
You can also continue looking at the homeworks and continue preparing for the next cohort. I guess

In [13]:
def retrieve_documents(query, index_name="course-questions", max_results=5):
    es = Elasticsearch("http://localhost:9200")
    
    search_query = {
        "size": max_results,
        "query": {
            "bool": {
                "must": {
                    "multi_match": {
                        "query": query,
                        "fields": ["question^3", "text", "section"],
                        "type": "best_fields"
                    }
                },
                "filter": {
                    "term": {
                        "course": "data-engineering-zoomcamp"
                    }
                }
            }
        }
    }
    
    response = es.search(index=index_name, body=search_query)
    documents = [hit['_source'] for hit in response['hits']['hits']]
    return documents

In [14]:
user_question = "How do I join the course after it has started?"

response = retrieve_documents(user_question)

for doc in response:
    print(f"Section: {doc['section']}\nQuestion: {doc['question']}\nAnswer: {doc['text']}\n\n")

Section: General course-related questions
Question: Course - Can I still join the course after the start date?
Answer: Yes, even if you don't register, you're still eligible to submit the homeworks.
Be aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute.


Section: General course-related questions
Question: Course - Can I still join the course after the start date?
Answer: Yes, even if you don't register, you're still eligible to submit the homeworks.
Be aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute.


Section: General course-related questions
Question: Course - Can I follow the course after it finishes?
Answer: Yes, we will keep all the materials after the course finishes, so you can follow the course at your own pace after it finishes.
You can also continue looking at the homeworks and continue preparing for the next cohort. I guess

In [15]:
context = ""

for doc in response:
    doc_str = f"Section: {doc['section']}\nQuestion: {doc['question']}\nAnswer: {doc['text']}\n\n"
    context += doc_str

context = context.strip()
print(context)

Section: General course-related questions
Question: Course - Can I still join the course after the start date?
Answer: Yes, even if you don't register, you're still eligible to submit the homeworks.
Be aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute.

Section: General course-related questions
Question: Course - Can I still join the course after the start date?
Answer: Yes, even if you don't register, you're still eligible to submit the homeworks.
Be aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute.

Section: General course-related questions
Question: Course - Can I follow the course after it finishes?
Answer: Yes, we will keep all the materials after the course finishes, so you can follow the course at your own pace after it finishes.
You can also continue looking at the homeworks and continue preparing for the next cohort. I guess y

In [16]:
prompt = f"""
You're a course teaching assistant. Answer the user QUESTION based on CONTEXT - the documents retrieved from our FAQ database. 
Only use the facts from the CONTEXT. If the CONTEXT doesn't contan the answer, return "NONE"

QUESTION: {user_question}

CONTEXT:

{context}
""".strip()

In [17]:
prompt

'You\'re a course teaching assistant. Answer the user QUESTION based on CONTEXT - the documents retrieved from our FAQ database. \nOnly use the facts from the CONTEXT. If the CONTEXT doesn\'t contan the answer, return "NONE"\n\nQUESTION: How do I join the course after it has started?\n\nCONTEXT:\n\nSection: General course-related questions\nQuestion: Course - Can I still join the course after the start date?\nAnswer: Yes, even if you don\'t register, you\'re still eligible to submit the homeworks.\nBe aware, however, that there will be deadlines for turning in the final projects. So don\'t leave everything for the last minute.\n\nSection: General course-related questions\nQuestion: Course - Can I still join the course after the start date?\nAnswer: Yes, even if you don\'t register, you\'re still eligible to submit the homeworks.\nBe aware, however, that there will be deadlines for turning in the final projects. So don\'t leave everything for the last minute.\n\nSection: General course-

In [18]:
from openai import OpenAI

client = OpenAI()

In [19]:
response = client.chat.completions.create(
    model="gpt-3.5-turbo",
    messages=[{"role": "user", "content": prompt}]
)
response

ChatCompletion(id='chatcmpl-9dlvBnmBpZ3ofnlMp1igeQneXOOXB', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content="You can still submit homeworks even if you don't register, but there will be deadlines for turning in the final projects. So don't leave everything for the last minute.", role='assistant', function_call=None, tool_calls=None))], created=1719266029, model='gpt-3.5-turbo-0125', object='chat.completion', service_tier=None, system_fingerprint=None, usage=CompletionUsage(completion_tokens=35, prompt_tokens=444, total_tokens=479))

In [20]:
answer = response.choices[0].message.content
print(answer)

You can still submit homeworks even if you don't register, but there will be deadlines for turning in the final projects. So don't leave everything for the last minute.


In [33]:
def build_context(documents):
    context = ""

    for doc in documents:
        doc_str = f"Section: {doc['section']}\nQuestion: {doc['question']}\nAnswer: {doc['text']}\n\n"
        context += doc_str
    
    context = context.strip()
    return context

role =  f"""
You're a course teaching assistant.
Answer the user QUESTION based on CONTEXT - the documents retrieved from our FAQ database.
Don't use other information outside of the provided CONTEXT.  
""".strip()

def build_prompt(user_question, documents):
    context = build_context(documents)
    return f"""
QUESTION: {user_question}

CONTEXT:

{context}
""".strip()

In [35]:
def ask_openai(prompt, model="gpt-3.5-turbo"):
    response = client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=[
            {"role": "system", "content": role},
            {"role": "user", "content": prompt}
        ]
    )
    answer = response.choices[0].message.content
    return answer

In [40]:
def qa_bot(user_question):
    context_docs = retrieve_documents(user_question)
    prompt = build_prompt(user_question, context_docs)
    answer = ask_openai(prompt)
    return answer

In [41]:
qa_bot("I'm getting invalid reference format: repository name must be lowercase")

'To resolve the "invalid reference format: repository name must be lowercase" issue in Docker when mounting volumes on Windows, you can try the following solutions:\n1. Move your data to a folder without spaces in the path.\n2. Replace the "-v" part with one of the following options:\n   -v /c:/some/path/ny_taxi_postgres_data:/var/lib/postgresql/data\n   -v //c:/some/path/ny_taxi_postgres_data:/var/lib/postgresql/data\n   -v /c/some/path/ny_taxi_postgres_data:/var/lib/postgresql/data\n   -v //c/some/path/ny_taxi_postgres_data:/var/lib/postgresql/data\n   --volume //driveletter/path/ny_taxi_postgres_data/:/var/lib/postgresql/data\n3. Try adding "winpty" before the whole command.\n4. Try adding quotes around the paths, such as:\n   -v "/c:/some/path/ny_taxi_postgres_data:/var/lib/postgresql/data"\n   -v "//c:/some/path/ny_taxi_postgres_data:/var/lib/postgresql/data"\n   -v "c:\\some\\path\\ny_taxi_postgres_data":/var/lib/postgresql/data\n5. Use the volume name instead of the path if othe

In [42]:
qa_bot("I can't connect to postgres port 5432, my password doesn't work")

'Based on the provided information, it seems that you are encountering an issue with connecting to PostgreSQL on port 5432 due to password authentication failure for user "root." One common reason for this issue could be that the port 5432 is already taken by another PostgreSQL instance, or the PostgreSQL service is running on your Windows machine.\n\nTo troubleshoot this issue, you can try the following steps:\n1. Substitute port 5432 with a different port (e.g., 5431) in your connection string.\n2. Check if there is a PostgreSQL service running on your Windows machine and stop it if it is running to resolve the port conflict.\n3. If the error persists, consider changing the PostgreSQL port to avoid conflicts.\n\nApplying these steps should help you resolve the password authentication failure issue when connecting to PostgreSQL on port 5432.'

In [43]:
qa_bot("how can I run kafka?")

'To run Kafka, you would typically need to set up and configure Kafka and its related components in your environment. For Confluent Kafka specifically, you can find the schema registry URL in Confluent Cloud by following these steps: \n- Go to Environment → default (or the environment name you specified) → Navigate to the right side bar → Click on “Stream Governance API” → Find the URL under “Endpoint” \nRemember to also create credentials from the Credentials section provided below the URL for authentication purposes.'